In [1]:
import zesje
from zesje.factory import create_app
from zesje.database import *

In [2]:
import logging
logging.basicConfig()
logging.getLogger('sqlalchemy.engine').setLevel(logging.INFO)

In [3]:
app = create_app(app_config={'DATA_DIRECTORY': 'data-dev'})
app.app_context().push()

In [4]:
db.drop_all()
db.create_all()
exam = Exam(name='Exam')
copy1 = Copy(number=1)
sub1 = Submission(exam=exam, copies=[copy1])
copy2, copy3 = Copy(number=2), Copy(number=3)
sub2 = Submission(exam=exam, copies=[copy2, copy3])
    
db.session.add(exam)
db.session.commit()

INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:PRAGMA main.table_info("student")
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:PRAGMA main.table_info("grader")
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:PRAGMA main.table_info("exam")
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:PRAGMA main.table_info("submission")
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:PRAGMA main.table_info("copy")
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:PRAGMA main.table_info("page")
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:PRAGMA main.table_info("problem")
INFO:sqlalchemy.engine.base.En

INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:COMMIT
INFO:sqlalchemy.engine.base.Engine:
CREATE TABLE submission (
	id INTEGER NOT NULL, 
	exam_id INTEGER NOT NULL, 
	student_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(exam_id) REFERENCES exam (id), 
	FOREIGN KEY(student_id) REFERENCES student (id)
)


INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:COMMIT
INFO:sqlalchemy.engine.base.Engine:
CREATE TABLE problem (
	id INTEGER NOT NULL, 
	name TEXT NOT NULL, 
	exam_id INTEGER NOT NULL, 
	grading_policy VARCHAR(11) DEFAULT 'set_blank' NOT NULL, 
	PRIMARY KEY (id), 
	FOREIGN KEY(exam_id) REFERENCES exam (id), 
	CONSTRAINT gradingpolicy CHECK (grading_policy IN ('set_nothing', 'set_blank', 'set_single'))
)


INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:COMMIT
INFO:sqlalchemy.engine.base.Engine:
CREATE TABLE scan (
	id INTEGER NOT NULL, 
	exam_id INTEGER NOT NULL, 
	name TEXT NOT NULL, 
	status TEXT NOT NULL, 

In [5]:
print(exam.copies)

INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.base.Engine:SELECT exam.id AS exam_id, exam.name AS exam_name, exam.token AS exam_token, exam.finalized AS exam_finalized, exam.grade_anonymous AS exam_grade_anonymous 
FROM exam 
WHERE exam.id = ?
INFO:sqlalchemy.engine.base.Engine:(1,)
INFO:sqlalchemy.engine.base.Engine:SELECT copy.id AS copy_id, copy.number AS copy_number, copy.submission_id AS copy_submission_id, copy.signature_validated AS copy_signature_validated 
FROM exam JOIN submission ON exam.id = submission.exam_id JOIN copy ON submission.id = copy.submission_id 
WHERE exam.id = ?
INFO:sqlalchemy.engine.base.Engine:(1,)


[<Copy 1>, <Copy 2>, <Copy 3>]


In [13]:
print(Exam.copies.order_by(Copy.number))

SELECT copy.id, copy.number, copy.submission_id, copy.signature_validated 
FROM exam, copy JOIN submission ON submission.id = copy.submission_id 
WHERE submission.exam_id = exam.id ORDER BY copy.number


In [7]:
print(Copy.query.select_from(Exam).\
               join(Exam.submissions).join(Submission.copies).filter(Exam.id == exam.id).all())

INFO:sqlalchemy.engine.base.Engine:SELECT copy.id AS copy_id, copy.number AS copy_number, copy.submission_id AS copy_submission_id, copy.signature_validated AS copy_signature_validated 
FROM exam JOIN submission ON exam.id = submission.exam_id JOIN copy ON submission.id = copy.submission_id 
WHERE exam.id = ?
INFO:sqlalchemy.engine.base.Engine:(1,)


[<Copy 1>, <Copy 2>, <Copy 3>]
